In [1]:
import tensorflow as tf
import numpy as np
from datetime import datetime   # date stamp the log directory
import json  # for saving and loading hyperparameters
import os, sys, re
import time

import absl
import absl.logging as logging
from tf2_models.matrix_caps import *
from util.config_util import get_model_params, get_task_params, get_train_params
from tf2_models.trainer import Trainer
from util.models import MODELS 
from util.tasks import TASKS
from notebook_utils import *

import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

import seaborn as sns; sns.set()

from tqdm import tqdm

gfile = tf.io.gfile
flags = absl.app.flags

[nltk_data] Downloading package punkt to /home/dehghani/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
class CapsConfig(object):
  def __init__(self,
               **kwargs):
    self.output_dim = 10
    self.A = 32
    self.B = 32
    self.C = 32
    self.D = 32
    self.epsilon = 1e-9
    self.l2 = 0.0000002
    self.final_lambda = 0.01
    self.iter_routing = 2

In [3]:
def spread_loss(scores, y, global_step):
    """Spread loss.

    "In order to make the training less sensitive to the initialization and 
    hyper-parameters of the model, we use “spread loss” to directly maximize the 
    gap between the activation of the target class (a_t) and the activation of the 
    other classes. If the activation of a wrong class, a_i, is closer than the 
    margin, m, to at then it is penalized by the squared distance to the margin."

    See Hinton et al. "Matrix Capsules with EM Routing" equation (3).

    Author:
    Ashley Gritzman 19/10/2018  
    Credit:
    Adapted from Suofei Zhang's implementation on GitHub, "Matrix-Capsules-
    EM-Tensorflow"
    https://github.com/www0wwwjs1/Matrix-Capsules-EM-Tensorflow  
    Args: 
    scores: 
      scores for each class 
      (batch_size, num_class)
    y: 
      index of true class 
      (batch_size, 1)  
    Returns:
    loss: 
      mean loss for entire batch
      (scalar)
    """
  
    batch_size = tf.shape(scores)[0]

    # margin = 0.2 + .79 * tf.sigmoid(tf.minimum(10.0, step / 50000.0 - 4))
    # where step is the training step. We trained with batch size of 64."
    m_min = 0.2
    m_delta = 0.79
    m = (m_min 
         + m_delta * tf.sigmoid(tf.minimum(10.0, global_step / 50000.0 - 4)))

    num_class = tf.shape(scores)[-1]

    y = tf.one_hot(y, num_class, dtype=tf.float32)

    # Get the score of the target class
    # (64, 1, 5)
    scores = tf.reshape(scores, shape=[batch_size, 1, num_class])
    # (64, 5, 1)
    y = tf.expand_dims(y, axis=2)
    # (64, 1, 5)*(64, 5, 1) = (64, 1, 1)
    at = tf.matmul(scores, y)

    # Compute spread loss, paper eq (3)
    loss = tf.math.square(tf.maximum(0., m - (at - scores)))

    # Sum losses for all classes
    # (64, 1, 5)*(64, 5, 1) = (64, 1, 1)
    # e.g loss*[1 0 1 1 1]
    loss = tf.matmul(loss, 1. - y)

    # Compute mean
    loss = tf.reduce_mean(loss)

    return loss


In [4]:
chkpt_dir='../tf_ckpts'
task_name='mnist'
strategy = tf.distribute.MirroredStrategy(devices=['/gpu:0','/gpu:1','/gpu:2','/gpu:3'])

#with strategy.scope():
task = TASKS[task_name](get_task_params(batch_size=32), data_dir='../data')
    
    
for x,y in task.train_dataset:
    print(x.shape, y.shape)
    break

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
(32, 28, 28, 1) (32,)


In [5]:
with strategy.scope():
    model = MatrixCaps(CapsConfig())
    example_x, example_y = next(iter(task.train_dataset))
    outputs = model(example_x, training=True)
    print(outputs.shape)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

    model.summary()

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


(32, 10)
Model: "matrix_caps"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo multiple                  4         
_________________________________________________________________
conv2d (Conv2D)              multiple                  832       
_________________________________________________________________
conv2d_1 (Conv2D)            multiple                  16896     
_________________________________________________________________
conv2d_2 (Conv2D)            multiple                  1056      
_________________________________________________________________
conv_caps1 (ConvCaps)        multiple                  147520    
_________________________________________________________________
conv_caps2 (ConvCaps)        multiple                  147520    
_________________________________________________________________
class_caps (FcCaps)          multiple         

In [ ]:
#with strategy.scope():
history = model.fit(task.train_dataset,
                    epochs=30,
                    steps_per_epoch=task.n_train_batches)

print('\nhistory dict:', history.history)

Epoch 1/30


INFO:tensorflow:batch_all_reduce: 17 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:batch_all_reduce: 17 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


...

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


1875/1875 [==============================] - 3534s 2s/step - sparse_categorical_accuracy: 0.6141 - loss: 1.9991
Epoch 2/30
1875/1875 [==============================] - 3521s 2s/step - sparse_categorical_accuracy: 0.9354 - loss: 1.8515
Epoch 3/30
1875/1875 [==============================] - 3519s 2s/step - sparse_categorical_accuracy: 0.9376 - loss: 1.8038
Epoch 4/30
1587/1875 [========================>.....] - ETA: 9:00 - sparse_categorical_accuracy: 0.9369 - loss: 1.7603